In [1]:
!pip install pandas numpy matplotlib seaborn wordcloud scikit-learn tensorflow scipy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 11.2 MB/s eta 0:00:0031m11.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 10.9 MB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 KB 10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.6 MB/s eta 0:00:00m eta 0:00:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5


In [4]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, silhouette_samples
import tensorflow as tf
import scipy.sparse as sp
import os
import warnings
warnings.filterwarnings('ignore')
# export TF_CPP_MIN_LOG_LEVEL=2
# export TF_ENABLE_ONEDNN_OPTS=0

In [5]:
# Enable TPU with diagnostics
print(f"TensorFlow version: {tf.__version__}")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if not tpu_address:
        raise ValueError("COLAB_TPU_ADDR environment variable not set. Ensure TPU runtime is selected.")
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + tpu_address)
    print(f"TPU endpoint: {tpu.cluster_spec().as_dict()}")
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("TPU initialized successfully")
except ValueError as e:
    print(f"TPU initialization failed: {e}")
    strategy = tf.distribute.get_strategy()

TensorFlow version: 2.19.0
TPU initialization failed: COLAB_TPU_ADDR environment variable not set. Ensure TPU runtime is selected.


In [ ]:
# Load and preprocess data in chunks
csv_file = '/TMDB_movie_dataset_v11.csv'  # Update with your local path

In [9]:
chunk_size = 100000
chunks = pd.read_csv(csv_file, chunksize=chunk_size, low_memory=False)
df_chunks = []

In [10]:
for chunk in chunks:
    chunk = chunk.fillna('')
    chunk['text'] = chunk.astype(str).agg(' '.join, axis=1)
    chunk['PrimaryGenre'] = chunk['genres'].str.split(',').str[0].str.strip()
    chunk['PrimaryGenre'] = chunk['PrimaryGenre'].replace('', 'Unknown')
    df_chunks.append(chunk)

In [11]:
df = pd.concat(df_chunks, ignore_index=True)
top_genres = df['PrimaryGenre'].value_counts().head(10).index
df['PrimaryGenre'] = df['PrimaryGenre'].where(df['PrimaryGenre'].isin(top_genres), 'Other')

In [19]:
# --- Load or compute TF-IDF matrix ---
tfidf_cache = 'tfidf_matrix.npz'
if os.path.exists(tfidf_cache):
    tfidf_matrix = sp.load_npz(tfidf_cache)
    print(f"Loaded cached TF-IDF matrix: {tfidf_matrix.shape}")
else:
    tfidf = TfidfVectorizer(stop_words='english', max_features=20000, min_df=5)
    tfidf_matrix = tfidf.fit_transform(df['text']).astype(np.float32)
    sp.save_npz(tfidf_cache, tfidf_matrix)
    print(f"Computed and saved TF-IDF matrix: {tfidf_matrix.shape}")
feature_names = tfidf.get_feature_names_out() if 'tfidf' in locals() else []

Loaded cached TF-IDF matrix: (1231678, 20000)


In [20]:
# --- Convert sparse matrix to tf.SparseTensor ---
def sparse_to_tensor(sparse_matrix):
    coo = sparse_matrix.tocoo()
    indices = np.vstack((coo.row, coo.col)).transpose()
    values = coo.data
    shape = coo.shape
    return tf.SparseTensor(indices=indices, values=values, dense_shape=shape)

In [21]:
# --- Compute cosine similarity in batches ---
def compute_cosine_similarity_local(tfidf_matrix, batch_size=1024):
    n_samples = tfidf_matrix.shape[0]
    tfidf_tensor = sparse_to_tensor(tfidf_matrix)
    tfidf_dense = tf.sparse.to_dense(tfidf_tensor)  # Convert to dense tensor

    def batch_cosine_sim(batch, matrix):
        norms_batch = tf.linalg.norm(batch, axis=1, keepdims=True)
        norms_matrix = tf.linalg.norm(matrix, axis=1, keepdims=True)
        normalized_batch = batch / (norms_batch + 1e-10)
        normalized_matrix = matrix / (norms_matrix + 1e-10)
        return tf.matmul(normalized_batch, normalized_matrix, transpose_b=True)

    sim_file = 'cosine_sim_chunks'
    os.makedirs(sim_file, exist_ok=True)
    chunk_index = 0

    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        batch = tfidf_dense[start:end]
        sim_batch = batch_cosine_sim(batch, tfidf_dense)
        np.save(f'{sim_file}/sim_chunk_{chunk_index}.npy', sim_batch.numpy())
        chunk_index += 1
        print(f"Saved similarity chunk {chunk_index} for rows {start} to {end}")

    cosine_sim = []
    for i in range(chunk_index):
        chunk = np.load(f'{sim_file}/sim_chunk_{i}.npy')
        cosine_sim.append(chunk)
    return np.concatenate(cosine_sim, axis=0)

In [22]:
# --- Compute and cache cosine similarity matrix ---
cosine_sim_cache = 'cosine_sim.npy'
if os.path.exists(cosine_sim_cache):
    cosine_sim = np.load(cosine_sim_cache)
    print(f"Loaded cached cosine similarity matrix: {cosine_sim.shape}")
else:
    cosine_sim = compute_cosine_similarity_local(tfidf_matrix)
    np.save(cosine_sim_cache, cosine_sim)
    print(f"Computed and saved cosine similarity matrix: {cosine_sim.shape}")

2025-05-29 17:15:28.952142: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (mklcpu) ran out of memory trying to allocate 91.77GiB (rounded to 98534240000)requested by op SparseToDense
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-05-29 17:15:28.952161: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for mklcpu
2025-05-29 17:15:28.952164: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-29 17:15:28.952165: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-29 17:15:28.952167: I externa

ResourceExhaustedError: {{function_node __wrapped__SparseToDense_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[1231678,20000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:SparseToDense] name: 

l/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (1024): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-29 17:15:28.952168: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (2048): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-29 17:15:28.952169: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (4096): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-29 17:15:28.952170: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (8192): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-29 17:15:28.952171: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (16384): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use

In [ ]:
# Content-based recommendation
def recommend_content(title: str, top_n: int = 5):
    idx = df[df['title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        return f"Movie '{title}' not found in dataset."
    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1:top_n+1]]
    recommendations = df.iloc[top_indices][['title', 'PrimaryGenre', 'production_companies', 'overview', 'popularity', 'tagline', 'keywords', 'runtime']].copy()
    recommendations['Similarity Score'] = [sim_scores[i][1] for i in range(1, top_n+1)]
    return recommendations

In [ ]:
# MiniBatchKMeans for faster clustering
num_clusters = 8
km = MiniBatchKMeans(n_clusters=num_clusters, random_state=42, batch_size=1000)
df['Cluster'] = km.fit_predict(tfidf_matrix)

In [ ]:
# Evaluate clustering with silhouette score
sil_score = silhouette_score(tfidf_matrix, df['Cluster'], metric='cosine', sample_size=2000, random_state=42)
print(f"Silhouette Score: {sil_score:.3f}")

In [ ]:
# Cluster summary
print("\nCluster Sizes and Top Genres:")
cluster_summary = df.groupby('Cluster').agg({
    'title': 'count',
    'PrimaryGenre': lambda x: x.value_counts().head(3).to_dict()
}).rename(columns={'title': 'Count'})
print(cluster_summary)

In [ ]:
# Sample movies per cluster
print("\nSample Movies per Cluster (Top 3 per cluster):")
for i in range(num_clusters):
    print(f"\nCluster {i}:")
    cluster_movies = df[df['Cluster'] == i][['title', 'PrimaryGenre', 'overview']].head(3)
    for _, row in cluster_movies.iterrows():
        print(f"- {row['title']} ({row['PrimaryGenre']}): {row['overview'][:100]}...")

In [ ]:
# Visualization 1: Silhouette Plot
silhouette_vals = silhouette_samples(tfidf_matrix, df['Cluster'], metric='cosine')
plt.figure(figsize=(10, 6))
y_lower, y_upper = 0, 0
for i in range(num_clusters):
    cluster_sil_vals = silhouette_vals[df['Cluster'] == i]
    cluster_sil_vals.sort()
    y_upper += len(cluster_sil_vals)
    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_sil_vals, alpha=0.7, label=f'Cluster {i}')
    y_lower += len(cluster_sil_vals)
plt.axvline(x=sil_score, color='red', linestyle='--', label='Avg Silhouette Score')
plt.title('Silhouette Plot for K-Means Clustering')
plt.xlabel('Silhouette Coefficient')
plt.ylabel('Cluster')
plt.legend()
plt.savefig('silhouette_plot.png')
plt.close()

In [ ]:
# Visualization 2: TruncatedSVD for 2D Visualization
svd = TruncatedSVD(n_components=2, random_state=42)
svd_result = svd.fit_transform(tfidf_matrix)
plt.figure(figsize=(12, 6))
sns.scatterplot(x=svd_result[:, 0], y=svd_result[:, 1], hue=df['Cluster'], palette='Set1', legend='full')
plt.title('Movie Clusters (TruncatedSVD Visualization)')
plt.xlabel('SVD 1')
plt.ylabel('SVD 2')
plt.savefig('svd_scatter.png')
plt.close()

In [ ]:
# Visualization 3: Genre Distribution
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='Cluster', hue='PrimaryGenre', palette='Set2')
plt.title('Genre Distribution Across Clusters')
plt.xlabel('Cluster')
plt.ylabel('Number of Movies')
plt.legend(title='Primary Genre', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('genre_distribution.png')
plt.close()

In [ ]:
# Visualization 4: Word Clouds for Each Cluster
for i in range(num_clusters):
    cluster_texts = ' '.join(df[df['Cluster'] == i]['text'])
    wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=30).generate(cluster_texts)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud for Cluster {i}')
    plt.axis('off')
    plt.savefig(f'wordcloud_cluster_{i}.png')
    plt.close()

In [ ]:
# Recommendation example
sample_movie = 'The Black Viper'
print(f"\nValidating Recommendations for '{sample_movie}':")
if df['title'].str.lower().eq(sample_movie.lower()).any():
    input_movie = df[df['title'].str.lower() == sample_movie.lower()][['title', 'PrimaryGenre', 'production_companies', 'overview']].iloc[0]
    print(f"Input Movie Details:\n- {input_movie['title']} (PrimaryGenre: {input_movie['PrimaryGenre']}, Production Companies: {input_movie['production_companies']}): {input_movie['overview'][:100]}...")
    recommendations = recommend_content(sample_movie, top_n=5)
    if isinstance(recommendations, str):
        print(recommendations)
    else:
        print("\nRecommended Movies:")
        print(recommendations[['title', 'PrimaryGenre', 'production_companies', 'Similarity Score', 'overview']].to_string(index=False))
else:
    print(f"Movie '{sample_movie}' not found in dataset.")

In [ ]:
# Visualization 5: Recommendation Bar Plot
idx = df[df['title'].str.lower() == sample_movie.lower()].index
if len(idx) > 0:
    idx = idx[0]
    sim_scores = cosine_sim[idx]
    top_indices = np.argsort(sim_scores)[::-1][1:6]
    top_scores = sim_scores[top_indices]
    top_titles = df.iloc[top_indices]['title'].values
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_scores, y=top_titles, palette='Blues_r')
    plt.title(f'Similarity Scores for Movies Similar to "{sample_movie}"')
    plt.xlabel('Cosine Similarity')
    plt.ylabel('Movie Title')
    plt.savefig('recommendation_bar_plot.png')
    plt.close()
else:
    print(f"Cannot plot similarity scores: '{sample_movie}' not found.")